## Importing and Preparing Source Data
- Read the data into memory as a Pandas DataFrame
- Profile the data for initial understanding and to identify flaws
- Address any issues and prepare for Exploratory Data Analysis (EDA)
- Prepare data for Machine Learning model development

### Import Required Libraries

In [1]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

#### Define a Function to Convert Column DataTypes

In [2]:
def set_datatypes(dframe):
    for col in dframe.columns:
        if dframe[col].dtypes.str.startswith('int'):
            dframe[col] = pd.to_numeric(dframe[col],
                                        downcast='integer',
                                        errors='coerce')
            
        elif dframe[col].dtypes.str.startswith('float'):
            dframe[col] = pd.to_numeric(dframe[col],
                                        downcast='float',
                                        errors='coerce')
            
        elif dframe[col].dtypes == 'object':
            dframe[col] = dframe[col].astype('category')
        
    return dframe

### Load the Data

In [3]:
data_dir = os.path.join(os.getcwd(), 'Data')
data_file = 'WA-Telco-Customer-Churn.csv';

# Read the source from the Web into a Pandas DataFrame.
working_file = os.path.join(data_dir, data_file)
df = pd.read_csv(working_file, header=0, index_col=0)

# Save a copy in case we screw something up!
raw_file = os.path.join(data_dir, 'WA-Telco-Customer-Churn-Raw.csv')
df.to_csv(raw_file)

# Make sure the index values are seed=1, increment=1
df.reset_index(drop=True, inplace=True)

### Profile the Data

In [4]:
# Display the number of rows and columns
print("Rows, Columns: ", df.shape)

# Display the first 5 observations
df.head()

Rows, Columns:  (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
# Inspect the unique values for each feature.
print("------------------------------------------------------------")
print("Number of Unique Customers:", len(df.customerID.unique()))
print("------------------------------------------------------------")

for col in df.columns:
    print("{0}: {1}".format(col, df[col].unique()))

------------------------------------------------------------
Number of Unique Customers: 7043
------------------------------------------------------------
customerID: ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
gender: ['Female' 'Male']
SeniorCitizen: [0 1]
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Con

In [6]:
# Inspect the data type of each column
print(df.dtypes)

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


##### Why does the TotalCharges column Contain Strings (object)?

In [7]:
# Convert the Text values to Floats!
df.TotalCharges = pd.to_numeric(df.TotalCharges, downcast='float', errors='coerce')
print(df.TotalCharges.dtypes)

float32


#### Check for Any Null or NaN Observations

In [8]:
df.isnull().any()

customerID          False
gender              False
SeniorCitizen       False
Partner             False
Dependents          False
tenure              False
PhoneService        False
MultipleLines       False
InternetService     False
OnlineSecurity      False
OnlineBackup        False
DeviceProtection    False
TechSupport         False
StreamingTV         False
StreamingMovies     False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges         True
Churn               False
dtype: bool

In [9]:
# TotalCharges appears to have missing values: How Many?
df.TotalCharges.isnull().sum()

11

In [10]:
# That's not too many, so impute 'NaN' with 0.00
df.TotalCharges.fillna(0.00, inplace=True)

### Prepare for Exploratory Data Analysis
- Capitalize Column Headers for Consistency
- Textual Values Produce More Readable Data Visualization Labels

In [11]:
for col in df.columns:
    if col[:1].islower():
        df = df.rename(columns={col : col.capitalize()})

In [12]:
# Convert SeniorCitizen to a textual feature.
df.loc[df.SeniorCitizen==1, ('SeniorCitizen')] = 'Yes'
df.loc[df.SeniorCitizen==0, ('SeniorCitizen')] = 'No'

df.head()

,Customerid,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,No,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.850000,No
1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.500000,No
2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.150002,Yes
3,7795-CFOCW,Male,No,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.750000,No
4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.649994,Yes


#### Save the Readable Feature-Set for EDA

In [13]:
dest_file = os.path.join(data_dir, 'WA-Telco-Customer-Churn-EDA.csv')
df.to_csv(dest_file)

### Prepare the Data for Machine Learning
- Machine Learning Algorithms Cannot Operate on Textual Data 
- Use LabelEncoding to Convert Textual Features into Numeric Values

#### Make Necessary Data Type Assignments for Label Encoding.

In [14]:
df = set_datatypes(df)

# Validate new data type assignments.
df.dtypes

Customerid          category
Gender              category
SeniorCitizen       category
Partner             category
Dependents          category
Tenure                 int64
PhoneService        category
MultipleLines       category
InternetService     category
OnlineSecurity      category
OnlineBackup        category
DeviceProtection    category
TechSupport         category
StreamingTV         category
StreamingMovies     category
Contract            category
PaperlessBilling    category
PaymentMethod       category
MonthlyCharges       float64
TotalCharges         float32
Churn               category
dtype: object

#### Encode Categorical Features

In [15]:
le = LabelEncoder()

for col in df.select_dtypes(include=['category']):
    df[col] = le.fit_transform(df[col])
    
df.dtypes

Customerid            int64
Gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
Tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges        float32
Churn                 int64
dtype: object

#### Inspect Converted Feature-Set

In [16]:
df.tail()

,Customerid,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7038,4853,1,0,1,1,24,1,2,0,2,...,2,2,2,2,1,1,3,84.80,1990.500000,0
7039,1525,0,0,1,1,72,1,2,1,0,...,2,0,2,2,1,1,1,103.20,7362.899902,0
7040,3367,0,0,1,1,11,0,1,0,2,...,0,0,0,0,0,1,2,29.60,346.450012,0
7041,5934,1,1,1,0,4,1,2,1,0,...,0,0,0,0,0,1,3,74.40,306.600006,1
7042,2226,1,0,0,0,66,1,0,1,2,...,2,2,2,2,2,1,0,105.65,6844.500000,0


#### Save Converted Features for Machine Learning Modeling

In [17]:
dest_file = os.path.join(data_dir, 'WA-Telco-Customer-Churn-ML.csv')
df.to_csv(dest_file)